In [171]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
import json

# Swedish COVID-19 data
The COVID-19 data is gathered from the Swedish Public Health agency

In [166]:
url = 'https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data'
f = requests.get(url)
infections_per_day = pd.read_excel(f.content,'Antal per dag region')
death_per_day = pd.read_excel(f.content,'Antal avlidna per dag')
week_data = pd.read_excel(f.content,'Veckodata Kommun_stadsdel')

In [174]:
with open('kommuner-kustlinjer.geo.json','r') as f:
    kommuner_kust = json.load(f)

In [197]:
def clean_data(df, week,):
    new_df = df.loc[df['veckonummer'] == week]
    new_df = new_df[['KnKod','nya_fall_vecka']]
    new_df['nya_fall_vecka'] = new_df['nya_fall_vecka'].str.replace('Okänd','0')
    new_df['nya_fall_vecka'] = new_df['nya_fall_vecka'].str.replace('<','')
    new_df['nya_fall_vecka'] = new_df['nya_fall_vecka'].astype(int)
    try:
        new_df = new_df.drop(new_df.loc[new_df['KnKod'] == 'Okänd'].index)
    except:
        pass
    return new_df

In [200]:
clean_df = clean_data(week_data, 31)

m = folium.Map(location = [62.135,9.8], zoom_start = 4)


m.choropleth(
    geo_data = kommuner_kust,
    data = clean_df,
    columns=['KnKod','nya_fall_vecka'],
    key_on='feature.properties.ref:se:kommun:kod',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=1,
    legend_name='Confirmed new cases',
    )

m


In [199]:
regions = infections_per_day.columns[2:]

@interact
def plot_per_region(region = regions):
    infections_per_day[region].plot()
    infections_per_day['Totalt_antal_fall'].plot()
    plt.legend()

interactive(children=(Dropdown(description='region', options=('Blekinge', 'Dalarna', 'Gotland', 'Gävleborg', '…